In [ ]:
# coding=utf-8

from math import sqrt
import time
import networkx as nx
import random
import numpy as np
import community
import time
import infomap
st=time.time()
class cada():
	def __init__(self, graph, algorithm='louvain', resolution=0.1):
		
		# First do community detection
		if algorithm == 'louvain':
			partition = community.best_partition(graph, resolution=resolution)
		else:
			partition = self.run_infomap(graph)
		
		communities = set()
		for node in graph.nodes():
			if node in partition:
				communities.add(partition[node])

		anom_score = {}

		# Compute anomaly score for each node
		for node in graph.nodes():
			comms = {}
			for neighbor in graph.neighbors(node):
				if neighbor != node:
					if partition[neighbor] not in comms:
						comms[partition[neighbor]] = 0

					comms[partition[neighbor]] += 1

			if len(comms) > 0:
				# The number of communities it is connected to. 
				comms = np.array(list(comms.values()))
				# print('nr communities connected', comms)
				max_com = np.max(comms)
				# print('Maxcommunity', max_com)
				comms = comms / max_com
				# print('Communities normalized', comms)
				anom_score[node] = np.sum(comms)		
				# print('Anomaly score., ', anom_score[node])

		self.anomaly_scores = sorted(anom_score.items(), key=lambda x: x[1])[::-1]


	def run_infomap(self, graph):
		"""
		Runs Infomap with infomap package 
		"""
		infomapSimple = infomap.Infomap("--two-level --silent")
		network = infomapSimple.network()
		
		for e in graph.edges():
			network.addLink(e[0], e[1])

		partition = {}
		infomapSimple.run();
		for node in infomapSimple.iterTree():
			if node.isLeaf():
				partition[node.physicalId] = node.moduleIndex()

		return partition

	def get_anomaly_scores(self, nr_anomalies=None):
		"""
		Returns tuple (node, anomaly_score) for either nr_anomalies or all
		"""
		if nr_anomalies:
			return self.anomaly_scores[:nr_anomalies]
		else:
			return self.anomaly_scores 

	def get_top_anomalies(self, nr_anomalies=100):
		"""
		Returns highest scoring anomalies
		"""					
		anomalies = []
		for anomaly in self.anomaly_scores[:nr_anomalies]:
			anomalies.append(anomaly[0])

		return anomalies

	def get_anomalies_threshold(self, threshold):
		"""
		Returns anomalies that are above a certain threshold.
		"""
		anomalies = []

		for anomaly in self.anomaly_scores:
			if anomaly[1] > threshold:
				anomalies.append(anomaly[0])
			else:
				break

		return anomalies
et=time.time()
print("Execution time:",et-st)


Execution time: 0.0005993843078613281


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
st=time.time()
g = nx.read_edgelist("/content/wiki.txt",create_using=nx.Graph(), nodetype = int)

# check if the data has been read properly or not.

nx.info(g)

# count the number of nodes

g.number_of_nodes()

# number of self-nodes

g.number_of_edges()
et=time.time()
print(et-st)


0.14073562622070312


In [ ]:
for u,v in g.edges():
      print ("("+str(u)+","+str(v)+")", g.get_edge_data(u,v))

(0,1) {}
(0,2) {}
(0,3) {}
(1,2) {}
(2,4) {}
(4,5) {}
(4,6) {}
(4,7) {}
(6,7) {}


In [ ]:
st=time.time()
obj = cada(g)
print(obj.get_anomalies_threshold(2))
et=time.time()
print(obj.get_top_anomalies())
obj.get_anomaly_scores()
print(et-st)

[1184, 285, 157, 719, 284, 279, 815, 718, 1892, 10, 3059, 286, 644, 292, 281, 1063, 4787, 23, 2405, 1493, 167, 1802, 910, 701, 875, 546, 340, 398, 1370, 1437, 69, 697, 2123, 888, 2425, 28, 22, 1106, 743, 1386, 680, 755, 640, 600, 774, 656, 2575, 2465, 1244, 1330, 874, 641, 2357, 915, 19, 1062, 29, 3598, 827, 2855, 2183, 665, 2381, 1088, 1420, 48, 2548, 1050, 1349, 1024, 310, 502, 937, 3150, 2099, 1021, 807, 1215, 1398, 239, 128, 5, 963, 896, 933, 857, 2838, 2204, 852, 1028, 262, 2191, 1795, 739, 891, 3701, 2765, 2427, 1937, 1486, 552, 3644, 2, 638, 78, 1664, 1463, 2648, 2348, 1961, 1256, 926, 880, 1052, 85, 1676, 1450, 1294, 2032, 3904, 2048, 1376, 1704, 906, 786, 339, 9, 373, 1056, 603, 4672, 604, 7, 277, 24, 3781, 1940, 3108, 2621, 862, 2291, 1519, 1094, 936, 1793, 133, 1064, 118, 254, 1471, 51, 1578, 221, 4226, 917, 855, 3347, 2607, 2190, 2172, 2004, 1835, 1770, 1415, 1358, 1761, 1631, 1259, 111, 903, 967, 1306, 159, 232, 1156, 3195, 2471, 1951, 860, 2761, 2758, 1968, 1960, 1956, 10

     |████████████████████████████████| 276kB 29.4MB/s 
  Created wheel for infomap: filename=infomap-1.5.1-cp37-cp37m-linux_x86_64.whl size=5637419 sha256=8ae7ed2311a8b3ddc5d4a3212f370b8ef059867d44fa48853f1dbfa183e4301a
  Stored in directory: /root/.cache/pip/wheels/23/46/2a/156d4e056221060a1f471a66e173c1a91143c7efa99037416f
Successfully built infomap
